**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
pip install scikit-video

     |████████████████████████████████| 2.3MB 2.9MB/s 


In [2]:
 pip install opencv-python

In [3]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import skvideo
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?


The function act implements the $\epsilon$-greedy algorithm. At time $t$, the agent select with probability $\epsilon$ an action randomly and with probability $1 - \epsilon$, he chooses with respect to the policy. 
So, the higher $\epsilon$ is, the more we encourage exploring. The lower $\epsilon$ is, the more we encourage exploiting.\
$\epsilon$ is essential because it makes the agent explore other possibilities during the training.


***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2 :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2 :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T    = 200
temperature  = 0.3
epochs_train = 15  # set small when debugging
epochs_test  = 15  # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

```position``` : the array has 3 states. 1 for the actual position of the rat on the grid, -1 where the rat can go and 0 where he can't go.

```board``` : it represents the reward of the grid cells. So the reward of cell choosen by the rat.

  


## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        '''
        Input:
        - s : état 
        Output : 
        - a : action
        '''
        
        a = np.random.randint(0, self.n_action, size=1)[0]
        return (a)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent , env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):

        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train = False)  #we are during the test

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            
        # Save as a mp4
        if e % 5 == 0:
            env.draw(prefix + str(e))
        #env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [10]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.0/12.0. Average score (-2.0)
Win/lose count 6.5/8.0. Average score (-1.75)
Win/lose count 12.5/24.0. Average score (-5.0)
Win/lose count 8.5/16.0. Average score (-5.625)
Win/lose count 8.0/17.0. Average score (-6.3)
Win/lose count 6.0/13.0. Average score (-6.416666666666667)
Win/lose count 12.5/9.0. Average score (-5.0)
Win/lose count 7.0/15.0. Average score (-5.375)
Win/lose count 6.5/14.0. Average score (-5.611111111111111)
Win/lose count 9.5/11.0. Average score (-5.2)
Win/lose count 11.0/9.0. Average score (-4.545454545454546)
Win/lose count 15.5/23.0. Average score (-4.791666666666667)
Win/lose count 13.0/17.0. Average score (-4.730769230769231)
Win/lose count 8.5/14.0. Average score (-4.785714285714286)
Win/lose count 10.5/12.0. Average score (-4.566666666666666)
Final score: -4.566666666666666


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




$\bf{\underline{ First \hspace{0.1cm} question :}} $

$$
Q^{\pi}(s,a) = E_{p^\pi}[\sum_{t\leq T} \gamma^{t} r(s_t,a_t)|s_0 = s, a_0 = a]\\
= E_{p^\pi}[  r(s,a) + \gamma\sum_{t=1}^{T} \gamma^{t-1} r(s_t,a_t)|s_0 = s, a_0 = a]\\
= r(s,a) + \gamma \sum_{s'}\sum_{a'}p(s_1 = s',a_1 = a'|s_0,a_0)E[\sum_{t=1}^{T}\gamma^{t-1}r(s_t,a_t)|s_1 = s', a_1 = a']\\
= r(s,a) + \gamma\sum_{s'}\sum_{a'}p(s',a'|s_0,a_0)Q^{\pi}(s',a')\\
=r(s,a) + \gamma E_{s',a'\sim p(.|s,a)}[Q^{\pi}(s',a')]
$$

$\bf{\underline{ Second \hspace{0.1cm} question :}} $

$$
Q^{*}(s,a) = \max_{\pi}E_{p^\pi}[\sum_{t\leq T} \gamma^{t} r(s_t,a_t)|s_0 = s, a_0 = a]\\
= \max_{\pi'} ( r(s,a) + \gamma \sum_{s'}\sum_{a'}p(s',a'|s_0,a_0)Q^{\pi'}(s',a')) \\
= r(s,a) + \gamma \sum_{s'}\sum_{a'}p(s',a'|s_0,a_0)\max_{\pi'}Q^{\pi'}(s',a') \\
= r(s,a) + \gamma \sum_{s'}\sum_{a'}p(s',a'|s_0,a_0)Q^{*}(s',a') \\
= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]
$$

$\bf{\underline{ Third \hspace{0.1cm} question :}}$
We can recognize the mean square error 
$$
\mathcal{L}(\theta)= E[(\hat{\theta} - \theta)^{2}]\\
E_{s' \sim \pi^*(.|s,a)}[\Vert Q^{*}(s,a,\theta^{*})-Q(s,a,\theta)\Vert^{2}\Vert^{2}]\\
E_{s' \sim \pi^*(.|s,a)}[\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}]
$$



***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            del self.memory[0]

    def random_access(self):
        i = np.random.choice( len(self.memory) )
        return self.memory[i]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.


In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        s = np.expand_dims(s,axis=0)
        a = np.argmax(self.model.predict(s))
        return a

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            
            state1, state2, action, reward, game_over = self.memory.random_access()
            target_q[i] = self.model.predict(np.expand_dims(state1, axis = 0))
            input_states[i,:,:,:] = state1
            state2 = np.expand_dims(state2, axis = 0)  
            
            if game_over_:
                target_q[i,action] = reward
                
            else:
                target_q[i,action] = reward + self.discount*np.max(self.model.predict(state2))
        
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(25,activation ='relu'))
        model.add(Dense(4))
        
        model.compile(adam(lr=lr, decay=1e-4), "mse")
        self.model = model
        

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))













Epoch 000/015 | Loss 0.2619 | Win/lose count 6.5/7.0 (-0.5)
Epoch 001/015 | Loss 0.8054 | Win/lose count 3.0/2.0 (1.0)
Epoch 002/015 | Loss 1.3416 | Win/lose count 5.5/4.0 (1.5)
Epoch 003/015 | Loss 1.5485 | Win/lose count 6.5/3.0 (3.5)
Epoch 004/015 | Loss 1.7216 | Win/lose count 8.5/6.0 (2.5)
Epoch 005/015 | Loss 1.6899 | Win/lose count 4.0/4.0 (0.0)
Epoch 006/015 | Loss 1.8797 | Win/lose count 4.5/1.0 (3.5)
Epoch 007/015 | Loss 1.8333 | Win/lose count 2.5/0 (2.5)
Epoch 008/015 | Loss 1.8046 | Win/lose count 2.5/1.0 (1.5)
Epoch 009/015 | Loss 1.8442 | Win/lose count 7.5/4.0 (3.5)
Epoch 010/015 | Loss 1.8868 | Win/lose count 6.5/2.0 (4.5)
Epoch 011/015 | Loss 2.0120 | Win/lose count 9.0/9.0 (0.0)
Epoch 012/015 | Loss 1.9469 | Win/lose count 8.0/2.0 (6.0)
Epoch 013/015 | Loss 1.9409 | Win/lose count 5.5/3.0 (2.5)
Epoch 014/015 | Loss 1.9410 | Win/lose count 2.5/4.0 (-1.5)


***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(50,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(30,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(adam(lr=lr, decay=1e-4), "mse")
        self.model = model

In [16]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/015 | Loss 2.1390 | Win/lose count 0.5/1.0 (-0.5)
Epoch 001/015 | Loss 2.1079 | Win/lose count 6.5/3.0 (3.5)
Epoch 002/015 | Loss 1.8665 | Win/lose count 15.0/9.0 (6.0)
Epoch 003/015 | Loss 1.9484 | Win/lose count 9.0/6.0 (3.0)
Epoch 004/015 | Loss 1.9087 | Win/lose count 7.0/3.0 (4.0)
Epoch 005/015 | Loss 1.8500 | Win/lose count 13.5/3.0 (10.5)
Epoch 006/015 | Loss 1.9044 | Win/lose count 13.5/6.0 (7.5)
Epoch 007/015 | Loss 1.9240 | Win/lose count 6.5/3.0 (3.5)
Epoch 008/015 | Loss 1.8043 | Win/lose count 4.0/2.0 (2.0)
Epoch 009/015 | Loss 1.8782 | Win/lose count 4.0/0 (4.0)
Epoch 010/015 | Loss 1.8313 | Win/lose count 5.5/3.0 (2.5)
Epoch 011/015 | Loss 1.8253 | Win/lose count 14.0/4.0 (10.0)
Epoch 012/015 | Loss 1.8517 | Win/lose count 6.0/4.0 (2.0)
Epoch 013/015 | Loss 1.9366 | Win/lose count 12.5/1.0 (11.5)
Epoch 014/015 | Loss 1.8515 | Win/lose count 17.5/3.0 (14.5)


***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [17]:
env = Environment(grid_size=size, max_time=T,temperature=0.5)
agent_cnn = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 3.0/0. Average score (3.0)
Win/lose count 1.5/0. Average score (2.25)
Win/lose count 0.5/0. Average score (1.6666666666666667)
Win/lose count 0.5/0. Average score (1.375)
Win/lose count 6.0/1.0. Average score (2.1)
Win/lose count 0.5/0. Average score (1.8333333333333333)
Win/lose count 1.0/0. Average score (1.7142857142857142)
Win/lose count 8.5/2.0. Average score (2.3125)
Win/lose count 0.5/0. Average score (2.111111111111111)
Win/lose count 0.5/0. Average score (1.95)
Win/lose count 0.5/0. Average score (1.8181818181818181)
Win/lose count 3.5/1.0. Average score (1.875)
Win/lose count 1.5/0. Average score (1.8461538461538463)
Win/lose count 0.5/0. Average score (1.75)
Win/lose count 2.0/0. Average score (1.7666666666666666)
Final score: 1.7666666666666666
Test of the FC
Win/lose count 0.5/1.0. Average score (-0.5)
Win/lose count 2.0/2.0. Average score (-0.25)
Win/lose count 0.5/1.0. Average score (-0.3333333333333333)
Win/lose count 1.0/1.0. Average scor

In [18]:
HTML(display_videos('cnn_test10.mp4'))

In [19]:
HTML(display_videos('fc_test10.mp4'))

| Temperature       |    score CNN    |      score  FC |
| :------------: | :-------------: | :-------------: |
| 0.1       |     1.06      |       -0.23  |
| 0.2       |     1.3       |        -0.16 |
| 0.3       |     2.1       |        -0.14 |
| 0.4       |     1.9       |        -0.13 |
| 0.5       |     2.8       |         0.1  |
| 0.6       |     2.13      |         0.3  |
| 0.7       |     3.26      |         0.6  |
| 0.8       |     4.63      |         1.2  |
| 0.9       |     6.83      |         2.2  |



On remarque 3 faits importants : \
 • Le score du CNN semble plus performant que celui du FC.\
 • Les scores semblent augmenter au fur et à mesure que la température augmente.\
 • A partir d'un moment, la souris finit par ne plus explorer la grille et fait des boucles. Cela se produit plus facilement lorsque le rat ne voit pas de récompense, c'est-à-dire que la température diminue.
 
L'architecture du CNN prend en partie en compte la spatialité de la grille en connectant les voisins d'une cellule aux mêmes poids. Plus la températur

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,decay_parameter_epsilon=0.3,prefix=''):
    #New training procedure that tries to improve the exploration of the algorithm
    #decay_parameter_epsilon in order to use the decreasing $\epsilon$-greedy exploration
    
    
    # Number of won games
    score = 0
    loss = 0
    decay_parameter_epsilon = 0.5
    print("epsilon",agent.epsilon)
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        if e % 5 == 0 and e!= 0:
            agent.set_epsilon(agent.epsilon*(decay_parameter_epsilon))
            print("epsilon",agent.epsilon)
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size)) #define maluses when going to a previously visited position
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train=False):
        """This function returns the new state, reward and decides if the
        game ends."""
        #During the training phase going back to a position where the rat have already been before tends to decrease the 
        #total reward hence there is this train parameter that we have added (it tries to enforce the exploration)

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position = np.zeros((self.grid_size,self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1     
        
        ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 1

        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        #At the begining the malus_position array must be setted to zero
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [0]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.5)
agent = DQN_CNN(size, lr=.1, epsilon = 0.9, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, 50, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

epsilon 0.9
Epoch 000/050 | Loss 1.4063 | Win/lose count 17.5/21.0 (-3.5)
Epoch 001/050 | Loss 2.0478 | Win/lose count 20.0/16.0 (4.0)
Epoch 002/050 | Loss 1.9163 | Win/lose count 18.0/12.0 (6.0)
Epoch 003/050 | Loss 2.1192 | Win/lose count 24.5/17.0 (7.5)
Epoch 004/050 | Loss 2.0720 | Win/lose count 16.5/16.0 (0.5)
epsilon 0.45
Epoch 005/050 | Loss 2.1792 | Win/lose count 13.0/10.0 (3.0)
Epoch 006/050 | Loss 1.9919 | Win/lose count 15.0/10.0 (5.0)
Epoch 007/050 | Loss 2.1410 | Win/lose count 16.5/7.0 (9.5)
Epoch 008/050 | Loss 1.8797 | Win/lose count 21.5/14.0 (7.5)
Epoch 009/050 | Loss 1.9649 | Win/lose count 12.5/18.0 (-5.5)
epsilon 0.225
Epoch 010/050 | Loss 1.9665 | Win/lose count 12.0/10.0 (2.0)
Epoch 011/050 | Loss 2.1402 | Win/lose count 18.5/11.0 (7.5)
Epoch 012/050 | Loss 1.9226 | Win/lose count 11.5/13.0 (-1.5)
Epoch 013/050 | Loss 1.8931 | Win/lose count 3.5/9.0 (-5.5)
Epoch 014/050 | Loss 1.9688 | Win/lose count 10.0/8.0 (2.0)
epsilon 0.1125
Epoch 015/050 | Loss 2.1476 | W

In [22]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 8.5/0. Average score (8.5)
Win/lose count 4.5/0. Average score (6.5)
Win/lose count 15.0/0. Average score (9.333333333333334)
Win/lose count 7.0/0. Average score (8.75)
Win/lose count 7.0/0. Average score (8.4)
Win/lose count 5.5/0. Average score (7.916666666666667)
Win/lose count 6.0/0. Average score (7.642857142857143)
Win/lose count 3.0/0. Average score (7.0625)
Win/lose count 20.5/0. Average score (8.555555555555555)
Win/lose count 13.5/0. Average score (9.05)
Win/lose count 4.0/0. Average score (8.590909090909092)
Win/lose count 7.0/1.0. Average score (8.375)
Win/lose count 2.0/0. Average score (7.884615384615385)
Win/lose count 13.0/1.0. Average score (8.178571428571429)
Win/lose count 15.0/0. Average score (8.633333333333333)
Final score: 8.633333333333333


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***